In [1]:
import os

os.getcwd()
os.chdir("../../")
os.getcwd()

'/data01/dl23vitcas/dl_project'

In [2]:
import sys
import argparse
import os

sys.argv = ["view", "--config", "config/single_task_object_detection.yaml"]

parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
args = parser.parse_args()

print(args.config)

config/single_task_object_detection.yaml


In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [4]:
W = torch.arange(12, dtype=torch.float32).reshape(4, 3)
b = torch.arange(4, dtype=torch.float32)
print(W)
print(b)

tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]])
tensor([0., 1., 2., 3.])


# SVD


In [5]:
U, S, V = torch.svd(W)

In [6]:
print(U)
print(S)
print(torch.diag(S))
print(V)

tensor([[-0.0835,  0.8325,  0.5319],
        [-0.3137,  0.4490, -0.8066],
        [-0.5438,  0.0656,  0.0176],
        [-0.7739, -0.3179,  0.2571]])
tensor([2.2447e+01, 1.4641e+00, 5.6755e-07])
tensor([[2.2447e+01, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.4641e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 5.6755e-07]])
tensor([[-0.4976, -0.7653, -0.4082],
        [-0.5740, -0.0624,  0.8165],
        [-0.6504,  0.6406, -0.4082]])


In [7]:
U @ torch.diag(S) @ V.t()  # deve essere un'approssimazione di W

tensor([[5.9605e-08, 1.0000e+00, 2.0000e+00],
        [3.0000e+00, 4.0000e+00, 5.0000e+00],
        [6.0000e+00, 7.0000e+00, 8.0000e+00],
        [9.0000e+00, 1.0000e+01, 1.1000e+01]])

# Truncated SVD


In [8]:
U, S, V = torch.svd(W)

In [9]:
t = 2
St = S[:t]
St_diag = torch.diag(St)

In [10]:
print(St)
print(St_diag)

tensor([22.4467,  1.4641])
tensor([[22.4467,  0.0000],
        [ 0.0000,  1.4641]])


In [11]:
# pad: left, right, up, down
# aggiunge un numero di colonne e righe di padding specificato  in questo caso solo una a destra e una sotto
St_diag = F.pad(input=St_diag, pad=(0, 1, 0, 1), mode="constant", value=0)
print(St_diag)

tensor([[22.4467,  0.0000,  0.0000],
        [ 0.0000,  1.4641,  0.0000],
        [ 0.0000,  0.0000,  0.0000]])


In [12]:
U @ St_diag @ V.t()  # deve essere un'approssimazione di W

tensor([[1.7881e-07, 1.0000e+00, 2.0000e+00],
        [3.0000e+00, 4.0000e+00, 5.0000e+00],
        [6.0000e+00, 7.0000e+00, 8.0000e+00],
        [9.0000e+00, 1.0000e+01, 1.1000e+01]])

To compress a network, the single fully connected layer corresponding to W is replaced by two fully connected layers, without a non-linearity between them. The first of these layers uses the weight matrix ΣtV T (and no biases) and the second uses U (with the original bi- ases associated with W ). This simple compression method gives good speedups when the number of RoIs is large.


# change weights matrix to FC layers


In [13]:
W = torch.arange(12, dtype=torch.float32).reshape(4, 3)
b = torch.arange(4, dtype=torch.float32)
fc = torch.nn.Linear(3, 4, bias=True)
fc.weight = torch.nn.Parameter(W)
fc.bias = torch.nn.Parameter(b)
print(fc.weight)
print(fc.bias)

Parameter containing:
tensor([[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.]], requires_grad=True)
Parameter containing:
tensor([0., 1., 2., 3.], requires_grad=True)


In [14]:
U, S, V = torch.svd(fc.weight)
t = 2
St = S[:t]
St_diag = torch.diag(St)
St_diag = F.pad(input=St_diag, pad=(0, 1, 0, 1), mode="constant", value=0)

In [15]:
desired_weights = St_diag @ V.t()
input_size = desired_weights.size(1)
output_size = desired_weights.size(0)
first_fc_layer = nn.Linear(input_size, output_size, bias=False)

with torch.no_grad():
    first_fc_layer.weight = nn.Parameter(desired_weights)

print("Pesi:", first_fc_layer.weight)
print("Bias:", first_fc_layer.bias)

Pesi: Parameter containing:
tensor([[-11.1689, -12.8838, -14.5986],
        [ -1.1205,  -0.0913,   0.9379],
        [  0.0000,   0.0000,   0.0000]], requires_grad=True)
Bias: None


In [16]:
desired_weights = U
desired_bias = fc.bias

input_size = desired_weights.size(1)
output_size = desired_weights.size(0)
fc_layer = nn.Linear(input_size, output_size)

with torch.no_grad():
    fc_layer.weight = nn.Parameter(desired_weights)
    fc_layer.bias = nn.Parameter(desired_bias)

# Stampa pesi e bias per verificare
print("Pesi:", fc_layer.weight)
print("Bias:", fc_layer.bias)

Pesi: Parameter containing:
tensor([[-0.0835,  0.8325,  0.5319],
        [-0.3137,  0.4490, -0.8066],
        [-0.5438,  0.0656,  0.0176],
        [-0.7739, -0.3179,  0.2571]], requires_grad=True)
Bias: Parameter containing:
tensor([0., 1., 2., 3.], requires_grad=True)


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def truncated_svd_decomposition(layer, t):
    """
    Applies truncated SVD decomposition on the given linear layer.

    Args:
    - layer (nn.Linear): The linear layer to be decomposed.
    - t (int): Number of singular values to keep.

    Returns:
    - nn.Sequential: A sequential model with two linear layers representing the truncated SVD.
    """
    # Perform SVD on the weight matrix
    W = layer.weight.data
    U, S, V = torch.svd(W)

    # Keep only the top t singular values
    U_t = U[:, :t]
    S_t = S[:t]
    V_t = V[:, :t]

    # Create the two new linear layers
    first_layer = nn.Linear(V_t.shape[0], t, bias=False)
    second_layer = nn.Linear(t, U_t.shape[0], bias=True)

    # Initialize the weights of the new layers
    first_layer.weight.data = (S_t.unsqueeze(1) * V_t.t()).t()
    second_layer.weight.data = U_t.t()

    # Set the bias of the second layer to be the same as the original layer
    second_layer.bias.data = layer.bias.data.clone()

    # Return a sequential model of the two layers
    return nn.Sequential(first_layer, second_layer)


# Example usage
original_layer = nn.Linear(4096, 4096)
compressed_layer = truncated_svd_decomposition(original_layer, t=256)

In [18]:
original_layer.weight[:20, :20]

tensor([[ 7.1849e-03,  3.7905e-03, -1.1864e-02,  6.9797e-03, -5.0416e-03,
          2.3502e-04,  1.8620e-03, -1.3436e-02, -1.2332e-02, -1.5618e-02,
          7.6030e-03,  4.2766e-03, -1.4970e-02,  2.1668e-03,  1.4707e-02,
          3.9622e-03, -9.1828e-03,  9.1545e-03, -1.4930e-02, -2.9909e-03],
        [-2.4309e-03, -1.6240e-03,  9.8420e-03, -1.3323e-02, -6.3613e-03,
          9.7429e-03, -4.2976e-03, -1.1442e-02,  7.0483e-03, -5.5832e-03,
         -5.4922e-03, -3.5614e-03, -4.0714e-03, -2.5810e-04, -1.3228e-03,
         -4.2175e-03, -5.9025e-04,  1.4878e-02, -4.6300e-03,  1.3859e-02],
        [-6.3232e-03,  4.9718e-03, -3.0489e-03,  5.1171e-03, -1.0547e-02,
          8.3346e-03,  1.1239e-02, -7.6160e-03, -4.8224e-04,  8.3424e-03,
         -1.3555e-02, -1.0539e-03,  5.3711e-03, -4.5070e-03,  2.2237e-03,
          1.5609e-02,  1.3556e-02,  1.8068e-03,  8.0590e-03,  1.3183e-02],
        [ 5.8181e-03,  1.7129e-03, -5.5282e-03, -7.2097e-03, -1.2825e-02,
          5.1431e-04, -9.8447e-03, 

In [19]:
from model import ObjectDetectionModel as ObjModelFull
from model_compressed import ObjectDetectionModel as ObjModelCompressed

In [20]:
def count_parameters(model):
    """
    Conta il numero di parametri nel modello.

    Args:
    - model (nn.Module): Il modello PyTorch di cui contare i parametri.

    Returns:
    - int: Il numero totale di parametri nel modello.
    """
    return sum(p.numel() for p in model.parameters())

In [21]:
count_parameters(ObjModelFull())

116065169

In [22]:
count_parameters(ObjModelCompressed())

67051738